In [1]:
import serial
import time
import csv

# Step 1: Connect to Arduino
try:
    arduino = serial.Serial(port='COM6', baudrate=115200, timeout=1)
    time.sleep(2)  # Wait for Arduino connection
    print(f"Connected to Arduino on {arduino.port}")
except serial.SerialException as e:
    print("Error: Could not open serial port!", e)
    exit()

# Step 2: Open CSV File for Storing Data
with open("eeg_data.csv", "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "EEG"])  # Column names

    print("Collecting EEG Data... Press Ctrl+C to stop.")

    try:
        while True:
            if arduino.in_waiting > 0:  
                data = arduino.readline().decode('utf-8').strip()
                if data.isdigit():  # Ensure valid number
                    timestamp = time.time()
                    writer.writerow([timestamp, data])
                    print(timestamp, data)
    except KeyboardInterrupt:
        print("\nStopping data collection.")
        arduino.close()
        print("Serial connection closed.")


Connected to Arduino on COM6
1741248106.3770428 494
1741248106.3790355 482
1741248106.3790355 495
1741248106.3800454 481
1741248106.3800454 499
1741248106.3800454 489
1741248106.3800454 504
1741248106.3810427 482
1741248106.3810427 506
1741248106.3820326 481
1741248106.3820326 508
1741248106.3820326 482
1741248106.3830383 512
1741248106.3830383 488
1741248106.3830383 514
1741248106.3830383 490
1741248106.3840384 517
1741248106.3840384 494
1741248106.3840384 518
1741248106.3850355 494
1741248106.3850355 504
1741248106.3850355 488
1741248106.3850355 501
1741248106.3860333 478
1741248106.3860333 492
1741248106.3860333 478
1741248106.3870392 493
1741248106.3870392 495
1741248106.3870392 506
1741248106.3880389 513
1741248106.3880389 513
1741248106.3880389 515
1741248106.3880389 509
1741248106.3890362 509
1741248106.3890362 505
1741248106.3890362 526
1741248106.3890362 525
1741248106.390038 532
1741248106.390038 528
1741248106.390038 541
1741248106.390038 527
1741248106.3910403 542
174124810

In [5]:
import pandas as pd
from scipy.signal import butter, filtfilt

# Step 1: Read the CSV file
df = pd.read_csv("eeg_data.csv")

# Ensure the 'EEG_Value' column exists and contains valid numeric data
if "EEG_Value" not in df.columns:
    raise ValueError("Column 'EEG_Value' not found in eeg_data.csv! Check your CSV format.")

df = df.dropna(subset=["EEG_Value"])  # Remove rows with missing values
df["EEG_Value"] = pd.to_numeric(df["EEG_Value"], errors="coerce")  # Convert to float, set errors to NaN
df = df.dropna()  # Remove rows where conversion failed

# Step 2: Apply Bandpass Filter (Remove noise)
def bandpass_filter(data, lowcut=0.5, highcut=30, fs=256, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    if low <= 0 or high >= 1:
        raise ValueError("Invalid frequency range. Adjust 'lowcut' and 'highcut'.")

    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

df["Filtered_EEG"] = bandpass_filter(df["EEG_Value"].values)

# Step 3: Save preprocessed data
df.to_csv("preprocessed_eeg.csv", index=False)
print("Preprocessing done! Data saved in preprocessed_eeg.csv")


Preprocessing done! Data saved in preprocessed_eeg.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Step 1: Load Preprocessed Data
df = pd.read_csv("preprocessed_eeg.csv")

# Step 2: Verify Required Columns
required_columns = ["Filtered_EEG", "EEG_Value"]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Missing column: {col} in dataset!")

# Step 3: Feature Extraction
df["Mean_Filtered_EEG"] = df["Filtered_EEG"].rolling(window=10, min_periods=1).mean()
df["Std_Filtered_EEG"] = df["Filtered_EEG"].rolling(window=10, min_periods=1).std()
df["Mean_EEG_Value"] = df["EEG_Value"].rolling(window=10, min_periods=1).mean()
df["Std_EEG_Value"] = df["EEG_Value"].rolling(window=10, min_periods=1).std()

# Drop NaN rows after feature extraction
df = df.dropna()

# Step 4: Generate Stress Levels (Since it is missing)
quantiles = df["Mean_Filtered_EEG"].quantile([0.33, 0.66])  # Divide into Low, Medium, High

def assign_stress_level(eeg_value):
    if eeg_value <= quantiles[0.33]:
        return "Low"
    elif eeg_value <= quantiles[0.66]:
        return "Medium"
    else:
        return "High"

df["Stress_Level"] = df["Mean_Filtered_EEG"].apply(assign_stress_level)  # Assign labels

# Step 5: Define Features and Labels
X = df[["Mean_Filtered_EEG", "Std_Filtered_EEG", "Mean_EEG_Value", "Std_EEG_Value"]].values
y = df["Stress_Level"].values  # Labels

# Step 6: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 7: Train Individual Models
knn = KNeighborsClassifier(n_neighbors=3)
svm = SVC(kernel="rbf", probability=True)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 8: Create Ensemble Model (Voting Classifier)
ensemble_model = VotingClassifier(
    estimators=[("knn", knn), ("svm", svm), ("rf", rf)],
    voting="hard"
)

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Step 9: Predict New EEG Signal
new_eeg = np.array([[10, 5, 8, 3]])  # Example EEG features (Mean & Std Dev of both signals)
predicted_class = ensemble_model.predict(new_eeg)

print(f"Predicted Stress Level: {predicted_class[0]}")
